## Contenct Creation at scale

- search for latesrt news: we will use GROK
- search and analyze market data - grok
- content creation
- create full blog post
- review blog


image.png


In [1]:
# Warning control
import warnings
import os, yaml

warnings.filterwarnings("ignore")

from crewai import Agent, Task, Crew
from dotenv import load_dotenv
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = os.getenv("OPENAI_MODEL_NAME")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ["GOOGLE_GEMINI"] = os.getenv("GOOLE_GEMINI")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

## Pydantic output


In [2]:
from pydantic import BaseModel, Field
from typing import List


class SocialMediaPost(BaseModel):
    platform: str = Field(
        ...,
        description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).",
    )
    content: str = Field(
        ...,
        description="The content of the social media post, including any hashtags or mentions.",
    )


class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(
        ..., description="A list of social media posts related to the article."
    )

## Reading File


In [3]:
# Define file paths for YAML configurations
files = {"agents": "config/agents_content.yaml", "tasks": "config/tasks_content.yaml"}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs["agents"]
tasks_config = configs["tasks"]

## LLM Model


In [4]:
# pip install -U litellm

In [5]:
import os
from langchain.chat_models import ChatLiteLLM  # ✅ use this instead of LiteLLM

# Load .env if you're using it
load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
os.environ["LITELLM_PROVIDER"] = "google"

# Initialize the model
from crewai import LLM

groq_llm = LLM(
    provider="google",
    model="gemini-pro",
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta",
    temperature=0.7,
)

## Agents and Tasks


# WebsiteSearchTool -> RAG as a tool

- agent search the internet: autommatically downlaod the web
- divide into chunks
- do Vector search to get the right info


In [6]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config["market_news_monitor_agent"],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    # llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config["data_analyst_agent"],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    # llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config["content_creator_agent"],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config["quality_assurance_agent"],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config["monitor_financial_news"], agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config["analyze_market_data"], agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config["create_content"],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task],
)

quality_assurance_task = Task(
    config=tasks_config["quality_assurance"],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput,
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent,
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task,
    ],
    verbose=True,
)

/Users/shankii/.pyenv/versions/3.11.9/lib/python3.11/site-packages/chromadb/db/impl/sqlite.py:111: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(
2025-05-18 19:04:09,869 - 8489667712 - sqlite.py-sqlite:111 - WARNING: ⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


## CREW AI


In [7]:
result = content_creation_crew.kickoff(
    inputs={
        "subject": "Inflation in the US and the impact on the stock market in 2025 after the trade tariffs were announced"
    }
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 00732e3d-e5ec-49dc-b2da-4f7e0c8e9e70                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 12b9a1a9-7ba9-4979-90e9-d8cb0f0e56b5
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 12b9a1a9-7ba9-4979-90e9-d8cb0f0e56b5
       Status: Executing Task...
    └── 🤖 Agent: Lead Market Analyst
        
            Status: In Progress

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2025 after the trade tariffs were announced. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2025 after the trade tariffs were announced.



🤖 Agent: Lead Market Analyst

    Status: In Progress
└── 🧠 Thinking...

...

# Agent: Chief Content Officer
## Task: Review and refine the content created on Inflation in the US and the impact on the stock market in 2025 after the trade tariffs were announced to ensure it meets the highest standards of accuracy, clarity, and brand alignment. Thoroughly proofread and edit the content, checking for errors, inconsistencies, and alignment with the brand voice. Ensure that the content accurately reflects the key insights and recommendations provided by the Data Analyst and Market News Monitor agents. Ensure that the final content is well-formatted in markdown, using appropriate headers, bullet points, links, and other markdown features to enhance readability and engagement.



# Agent: Chief Content Officer
## Final Answer: 
{
  "article": "# Comprehensive Analysis Report: The Impact of Inflation and Trade Tariffs on the U.S. Stock Market in 2025\n\n## Executive Summary\nThis report highlights the critical intersections of inflationary pressures, trade tariffs, and 

🚀 Crew: crew
├── 📋 Task: 12b9a1a9-7ba9-4979-90e9-d8cb0f0e56b5
│      Assigned to: Lead Market Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Lead Market Analyst
│       
│           Status: ✅ Completed
├── 📋 Task: 55ee0d84-e046-47f5-a9af-f40f9c43ebc3
│      Assigned to: Chief Data Strategist
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Chief Data Strategist
│       
│           Status: ✅ Completed
├── 📋 Task: 1f25fd7e-becf-4552-8271-36bf5604b8c1
│      Assigned to: Creative Content Director
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Creative Content Director
│       
│           Status: ✅ Completed
└── 📋 Task: cdbdec31-8abb-4169-8607-8ea6c38bf92c
       Status: Executing Task...
    └── 🤖 Agent: Chief Content Officer
        
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 12b9a1a9-7ba9-4979-90e9-d8cb0f0e56b5
│      Assigned to: Lead Market Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Lead Market Analyst
│       
│           Status: ✅ Completed
├── 📋 Task: 55ee0d84-e046-47f5-a9af-f40f9c43ebc3
│      Assigned to: Chief Data Strategist
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Chief Data Strategist
│       
│           Status: ✅ Completed
├── 📋 Task: 1f25fd7e-becf-4552-8271-36bf5604b8c1
│      Assigned to: Creative Content Director
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Creative Content Director
│       
│           Status: ✅ Completed
└── 📋 Task: cdbdec31-8abb-4169-8607-8ea6c38bf92c
       Assigned to: Chief Content Officer
    
       Status: ✅ Completed
    └── 🤖 Agent: Chief Content Officer
        
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cdbdec31-8abb-4169-8607-8ea6c38bf92c                                                                     │
│  Agent: Chief Content Officer                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 00732e3d-e5ec-49dc-b2da-4f7e0c8e9e70                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Social content


In [15]:
import textwrap

posts = result.pydantic.model_dump()["social_media_posts"]
for post in posts:
    platform = post["platform"]
    content = post["content"]
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print("-" * 50)

Twitter
Is the U.S. stock market prepared for the impact
of rising inflation in 2025 due to trade tariffs?
Stay informed to make savvy investment decisions!
#InvestmentInsights #Inflation2025
--------------------------------------------------
LinkedIn
As inflation projections climb in the wake of
trade tariffs, investors must strategize for
increased volatility in the stock market. Discover
how to navigate this financial landscape with our
latest insights. [Link to full report]
--------------------------------------------------


In [10]:
from IPython.display import display, Markdown

display(Markdown(result.pydantic.dict()["article"]))

/var/folders/rl/67j4t3550q16y6tcnnp69_jw0000gn/T/ipykernel_48780/2496794582.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  display(Markdown(result.pydantic.dict()["article"]))


# Comprehensive Analysis Report: The Impact of Inflation and Trade Tariffs on the U.S. Stock Market in 2025

## Executive Summary
This report highlights the critical intersections of inflationary pressures, trade tariffs, and their implications for the U.S. stock market in 2025. It synthesizes trends, responds to current data insights from multiple financial institutions, and provides recommendations for content creation that resonate with investor sentiment.

## 1. **U.S. Stock Performance and Tariffs**  
- The S&P 500 index has shown resilience, nearing record highs but still 3% lower than its earlier peak. This performance is largely influenced by President Trump’s trade policies, which initiated a trade war particularly affecting relations with Canada and Mexico.  
   - Source: [Fortune](https://fortune.com)

## 2. **Inflation Projections and Economic Impact**  
- Analysts predict an increase in price levels by 2.3% as a short-term effect of tariffs. This equates to an average household consumer loss approximating $3,800.  
- Projections indicate that the Personal Consumption Expenditures (PCE) price inflation may escalate to 2.7% by 2025.  
   - Source: Yale Budget Lab; JPMorgan  

### **Potential Impacts on Consumer Behavior**  
- Rising inflation is expected to adjust consumer spending and reshape investment strategies across sectors.

## 3. **Federal Reserve's Concerns**  
- Federal Reserve Chair Jerome Powell has flagged tariffs as unprecedented challenges. This raises concerns about stock market volatility due to trade tensions.  
   - Source: [CNN](https://cnn.com)

## 4. **Market Recovery and Future Outlook**  
- Despite recent rebounds, experts caution about potential slow gains in the stock market, influenced by shifting inflation expectations.  
   - Source: Reuters; PBS  
- Investors must brace for heightened market volatility, adjusting strategies to navigate uncertainty.

## 5. **Corporate and Investor Sentiment**  
- Ongoing uncertainties surrounding tariffs may dampen corporate capital expenditure, resulting in cautious investor behavior.  
   - Source: [BlackRock](https://www.blackrock.com)

### **Investor Action Points**  
- Investors should remain vigilant, staying updated about economic indicators and regulatory shifts that may dictate market trajectories.

## 6. **Overall Implications for Content Strategy**  
- The intersection of inflationary pressures and various trade policies presents rich content opportunities.  
- By delivering consistently updated analyses on market trends associated with tariffs and inflation, audiences can engage with actionable insights that enhance financial resilience.  
- Timely monitoring of economic indicators and regulatory changes will be necessary for providing pertinent information to investors and stakeholders.

## Data Visualizations  
- **Inflation Trends Over Time**: A graphical representation showcasing the projected inflation rates and corresponding market impact.  
- **Tariff Effects on PCE Prices**: A chart illustrating the direct correlation between elevated tariffs and consumer price index fluctuations.

### **Recommendations for Content Development**  
1. **Regular Updates**: Maintain a constant flow of information regarding inflation rates and tariff impacts on market dynamics.  
2. **Educational Content**: Provide educational resources aimed at refining investor acumen in response to economic pressures.  
3. **Interactive Platforms**: Leverage media channels to create interactive content that allows consumers to understand and anticipate market movements.

## Conclusion  
As inflation continues to bellyache the economic landscape, the repercussions of tariffs substantially affect the U.S. stock market. This synthesis not only delineates the current state but also anticipates future trajectories that inform strategic investment decisions. Establishing a content strategy around these findings will serve to navigate the evolving financial terrain effectively.

--- 

**By synthesizing extensive market data and incorporating these actionable insights, investors can develop strategies to thrive in an increasingly volatile economic landscape.**

----------

---

# Navigating the Financial Landscape: The Impact of Inflation and Trade Tariffs on the U.S. Stock Market in 2025 

## Overview:
In 2025, the landscape for investors has become increasingly complex due to rising inflation levels attributable to ongoing trade tariffs. This report delves into the implications of these economic dynamics on the U.S. stock market, offering valuable insights for investors looking to navigate this shifting terrain.

### 1. **Current Market Performance Amid Tariffs**  
- The **S&P 500** index is currently nearing record highs yet remains about **3% below its peak** earlier this year. This performance is predominantly shaped by **President Trump's trade policies** that sparked a trade war focusing initially on Canada and Mexico. [Source: Fortune](https://fortune.com)

### 2. **Inflation Projections and Economic Consequences**  
- Short-term forecasts suggest a **2.3% increase** in price levels due to tariffs, translating to significant consumer losses averaging **$3,800** per household. It is projected that the **Personal Consumption Expenditures (PCE)** price inflation will hit **2.7% by 2025**. [Source: Yale Budget Lab; JPMorgan]
  
### 3. **Federal Reserve Insights**  
- **Federal Reserve** Chair **Jerome Powell** has cautioned that current tariffs pose unprecedented challenges, hinting at potential market volatility as investors react to trade tensions. This underscores the need for investors to remain vigilant in monitoring evolving economic indicators. [Source: CNN]

### 4. **Market Recovery Ambiguity**  
- Despite recent rebounds in the market, experts suggest that gains may slow amid **shifting inflation expectations**. Investors are advised to prepare for **heightened volatility** and to adjust their strategies in light of the evolving situation. [Source: Reuters; PBS]

### 5. **Investor Sentiment and Corporate Spending**  
- The uncertainty stemming from tariffs is likely to stifle **corporate capital spending**, leading to cautious investor behavior. Therefore, it is essential for investors to stay abreast of the latest economic developments. [Source: BlackRock]

## Multimedia Elements

### Graphical Representation: **Projected Inflation Trends**  
![Projected Inflation Trends](https://example.com/projected_inflation_trends)

### Infographic: **Impact of Tariffs on the Stock Market**  
![Tariff Impact Infographic](https://example.com/tariff_impact_infographic)

### 6. **Strategic Recommendations for Investors**  
1. **Stay Informed**: Continuously seek out updates regarding inflation rates and the resulting impact on market dynamics.  
2. **Educational Resources**: Look for educational content that aids in refining your investment strategies amidst economic pressures.  
3. **Utilize Interactive Media**: Engage with platforms that offer interactive content designed to enhance understanding of market fluctuations.  

## Conclusion  
As inflationary pressures intensify due to trade policies, understanding their implications on stock market performance is crucial for investors. By implementing a content strategy informed by these findings, stakeholders can effectively prepare for the challenges and opportunities that lie ahead. Keeping a pulse on economic indicators will be imperative for successfully navigating this fluctuating financial landscape.

--- 